In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import scipy as sp
from wordcloud import WordCloud, STOPWORDS
import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))
        
import surprise
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import train_test_split
import time
from surprise import SVD
from surprise import accuracy
import re
import plotly.express as px
import scipy as sp
from wordcloud import WordCloud, STOPWORDS
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise.accuracy import rmse
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline  import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures, Normalizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.manifold import TSNE
from catboost import CatBoostRegressor

# Packages for model evaluation
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics.pairwise import linear_kernel
from sklearn.decomposition import PCA
from time import time

# Package to suppress warnings
import warnings
warnings.filterwarnings("ignore")

# Packages for saving models
import pickle

In [4]:

sample_submission = pd.read_csv('/kaggle/input/edsa-movie-recommendation-2022/sample_submission.csv')
movies_df = pd.read_csv('/kaggle/input/edsa-movie-recommendation-2022/movies.csv')
imdb_df = pd.read_csv('/kaggle/input/edsa-movie-recommendation-2022/imdb_data.csv')
genome_score = pd.read_csv('/kaggle/input/edsa-movie-recommendation-2022/genome_scores.csv')
genome_tags = pd.read_csv('/kaggle/input/edsa-movie-recommendation-2022/genome_tags.csv')
train_df = pd.read_csv('/kaggle/input/edsa-movie-recommendation-2022/train.csv')
test_df = pd.read_csv('/kaggle/input/edsa-movie-recommendation-2022/test.csv')
tags_df = pd.read_csv('/kaggle/input/edsa-movie-recommendation-2022/tags.csv')
links_df = pd.read_csv('/kaggle/input/edsa-movie-recommendation-2022/links.csv')

In [6]:
#Join df_imdb and df_movies datasets 
imbd_movie = imdb_df.merge(movies_df, on='movieId')
#Show text data to be used for analysis 
imbd_movie[['title', 'title_cast', 'director', 'plot_keywords', 'genres']].head()

In [7]:
#Fill NaN for each value with a corresponding filling string
imbd_movie['title_cast'].fillna('no title_cast listed',inplace=True)
imbd_movie['plot_keywords'].fillna('no plot_keywords listed ',inplace=True)
imbd_movie['director'].fillna('no director listed',inplace=True)

In [8]:
#Initiatlise the empty list 'plot_key'
plot_key = []
#Iterate through the 'plot_keywords' column
for i in imbd_movie['plot_keywords']:
#Split the plot keyword on the '|' character and append the list of words to the empty list
    plot_key.append(i.split('|'))

In [9]:
#Update the column with the new plot_keys seperated into a list
imbd_movie['plot_keywords'] = plot_key

In [10]:
#Initiatlise the empty list 'title_cast'
title_cast = []
#Iterate through the 'title_cast'' column
for i in imbd_movie['title_cast']:
#Split the plot keyword on the '|' character and append the list of words to the empty list
    title_cast.append(i.split('|'))

In [11]:
#Update the column with the new title_cast seperated into a list
imbd_movie['title_cast'] = title_cast

In [12]:
#Initiatlise the empty list 'genre_list'
genre_list = []
#Iterate through the 'genres' column
for i in imbd_movie['genres']:
#Split the plot keyword on the '|' character and append the list of words to the empty list
    genre_list.append(i.split('|'))

In [13]:
#Update the column with the new genre seperated into a list
imbd_movie['genres'] = genre_list

In [14]:
#View of reworked dataset
imbd_movie[['title', 'title_cast', 'director', 'plot_keywords', 'genres']].head()

In [15]:
def string_function(x):
    """merges name and surname into one name"""
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [16]:
features = ['title_cast','director','plot_keywords','genres']

for feature in features:
    imbd_movie[feature] = imbd_movie[feature].apply( string_function)

In [17]:
def create_sentence(x):
    return ' '.join(x['plot_keywords']) + ' ' + ' '.join(x['title_cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])
imbd_movie['sentence'] = imbd_movie.apply(create_sentence, axis=1)

In [18]:
imbd_movie.head()

In [19]:
#  Create the count matrix
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(imbd_movie['sentence'])

In [20]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [21]:
imbd_movie = imbd_movie.reset_index()
indices = pd.Series(imbd_movie.index, index=imbd_movie['title'])

In [22]:
def get_recommendations(title, cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return imbd_movie['title'].iloc[movie_indices]

In [23]:
# Get similar movies 
get_recommendations('American Beauty (1999)', cosine_sim)

In [24]:
# Load the 1M dataset
train_df = train_df.drop('timestamp', axis=1)
train_subset = train_df[:1000000]
reader = Reader(rating_scale=(train_df['rating'].min(), train_df['rating'].max()))
data = Dataset.load_from_df(train_df[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=42)

In [23]:
nmf_model = NMF(n_epochs=50, n_factors=200, random_state=42,verbose=True)
nmf_model.fit(trainset)
nmf_predictions =nmf_model.test(testset)
nmf_rmse = accuracy.rmse(nmf_predictions)

In [24]:
cc_model = CoClustering(n_epochs=50,random_state=42)
cc_model.fit(trainset)
cc_predictions = cc_model.test(testset)
cc_rmse=accuracy.rmse(cc_predictions)

In [25]:
svd_model = SVD(n_epochs=50,n_factors=400,init_std_dev=0.01,random_state=42,verbose=True)
svd_model.fit(trainset)
svd_predictions = svd_model.test(testset)
svd_rmse = accuracy.rmse(svd_predictions)

In [26]:
# Hypertuning Singular Value Decomposition
trainset, testset = train_test_split(data, test_size=0.01, random_state=42)

# Modelling of the SVD hypertuning
svd_algo_hyper = SVD(lr_all=0.0085,
                     reg_all=0.01,
                     n_epochs=30,
                     init_std_dev=0.01,
                     random_state= 100)
svd_algo_hyper.fit(trainset)

# Predicting on the validation set
svd_hyper_predictions = svd_algo_hyper.test(testset)

# Convert the predictions to dataframe
accuracy.rmse(svd_hyper_predictions)

In [26]:
pred_svd = [svd_model.predict(row.userId,row.movieId) for idx,row in test_df.iterrows()]

# Converting the predictions to a dataframe
test_pred_svd = pd.DataFrame(pred_svd)

In [27]:
# Concatenate each userId and movieId into a single Id column for submission
test_pred_svd['Id'] =  test_pred_svd['userId'].astype(str).str.zfill(1) + '_' + test_pred_svd['movieId'].astype(str).str.zfill(1)

In [ ]:
svd_predictions = test_pred_svd[['Id','rating']]
svd_predictions.head()

In [ ]:
svd_predictions.to_csv('./svd_pred_submission.csv', index=False)